In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.random.set_seed(777)  # for reproducibility
print(tf.__version__)

2.8.0


In [2]:
# Checkpoint function
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
# Data load & pre-processing function
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
# Performance function
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [10]:
# Model function
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate) :
    return tf.keras.layers.Dropout(rate) # dropout

In [11]:
# Create model (class version)
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=0.5)) # dropout

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [12]:
# Create model (function version)
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())
        model.add(dropout(rate=0.5))

    model.add(dense(label_dim, weight_init))

    return model

In [13]:
# Define data & hyper-parameter
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [14]:
# Define model & optimizer & writer
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [15]:
# Restore checkpoint & start train or test phase
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.6381, train_loss: 2.36959839, train_accuracy: 0.2344, test_Accuracy: 0.1634
Epoch: [ 0] [    1/  468] time: 0.8949, train_loss: 2.35990191, train_accuracy: 0.3750, test_Accuracy: 0.2774
Epoch: [ 0] [    2/  468] time: 1.3397, train_loss: 2.32028866, train_accuracy: 0.3984, test_Accuracy: 0.3327
Epoch: [ 0] [    3/  468] time: 1.6671, train_loss: 2.26515079, train_accuracy: 0.3750, test_Accuracy: 0.3938
Epoch: [ 0] [    4/  468] time: 1.9356, train_loss: 2.26758432, train_accuracy: 0.5234, test_Accuracy: 0.4235
Epoch: [ 0] [    5/  468] time: 2.1799, train_loss: 2.27882576, train_accuracy: 0.4297, test_Accuracy: 0.4251
Epoch: [ 0] [    6/  468] time: 2.4420, train_loss: 2.25754642, train_accuracy: 0.4141, test_Accuracy: 0.4453
Epoch: [ 0] [    7/  468] time: 2.6892, train_loss: 2.15085888, train_accuracy: 0.4297, test_Accuracy: 0.4742
Epoch: [ 0] [    8/  468] time: 2.9325,

Epoch: [ 0] [   74/  468] time: 19.5633, train_loss: 0.79151064, train_accuracy: 0.8438, test_Accuracy: 0.8885
Epoch: [ 0] [   75/  468] time: 19.8095, train_loss: 0.78586936, train_accuracy: 0.8516, test_Accuracy: 0.8894
Epoch: [ 0] [   76/  468] time: 20.0887, train_loss: 0.65138537, train_accuracy: 0.8672, test_Accuracy: 0.8913
Epoch: [ 0] [   77/  468] time: 20.3322, train_loss: 0.56848371, train_accuracy: 0.8750, test_Accuracy: 0.8918
Epoch: [ 0] [   78/  468] time: 20.6004, train_loss: 0.51681614, train_accuracy: 0.9062, test_Accuracy: 0.8929
Epoch: [ 0] [   79/  468] time: 20.8488, train_loss: 0.59066987, train_accuracy: 0.8750, test_Accuracy: 0.8937
Epoch: [ 0] [   80/  468] time: 21.0907, train_loss: 0.55488056, train_accuracy: 0.8594, test_Accuracy: 0.8934
Epoch: [ 0] [   81/  468] time: 21.3336, train_loss: 0.49785495, train_accuracy: 0.9297, test_Accuracy: 0.8941
Epoch: [ 0] [   82/  468] time: 21.5809, train_loss: 0.60823190, train_accuracy: 0.8906, test_Accuracy: 0.8968
E

Epoch: [ 0] [  148/  468] time: 38.5164, train_loss: 0.39079526, train_accuracy: 0.9297, test_Accuracy: 0.9159
Epoch: [ 0] [  149/  468] time: 38.7611, train_loss: 0.38038355, train_accuracy: 0.9062, test_Accuracy: 0.9162
Epoch: [ 0] [  150/  468] time: 39.0215, train_loss: 0.39744502, train_accuracy: 0.9375, test_Accuracy: 0.9165
Epoch: [ 0] [  151/  468] time: 39.2948, train_loss: 0.29324546, train_accuracy: 0.9531, test_Accuracy: 0.9196
Epoch: [ 0] [  152/  468] time: 39.5454, train_loss: 0.42189044, train_accuracy: 0.9141, test_Accuracy: 0.9222
Epoch: [ 0] [  153/  468] time: 39.8162, train_loss: 0.37719217, train_accuracy: 0.9453, test_Accuracy: 0.9247
Epoch: [ 0] [  154/  468] time: 40.1019, train_loss: 0.38563699, train_accuracy: 0.9219, test_Accuracy: 0.9238
Epoch: [ 0] [  155/  468] time: 40.3740, train_loss: 0.25016993, train_accuracy: 0.9453, test_Accuracy: 0.9214
Epoch: [ 0] [  156/  468] time: 40.6323, train_loss: 0.31321424, train_accuracy: 0.9375, test_Accuracy: 0.9186
E

Epoch: [ 0] [  222/  468] time: 58.9467, train_loss: 0.25265989, train_accuracy: 0.9531, test_Accuracy: 0.9382
Epoch: [ 0] [  223/  468] time: 59.2159, train_loss: 0.22818357, train_accuracy: 0.9453, test_Accuracy: 0.9408
Epoch: [ 0] [  224/  468] time: 59.5577, train_loss: 0.43268892, train_accuracy: 0.9062, test_Accuracy: 0.9420
Epoch: [ 0] [  225/  468] time: 59.8084, train_loss: 0.34220973, train_accuracy: 0.9297, test_Accuracy: 0.9432
Epoch: [ 0] [  226/  468] time: 60.0700, train_loss: 0.31045723, train_accuracy: 0.9297, test_Accuracy: 0.9425
Epoch: [ 0] [  227/  468] time: 60.3233, train_loss: 0.33742589, train_accuracy: 0.9609, test_Accuracy: 0.9405
Epoch: [ 0] [  228/  468] time: 60.5745, train_loss: 0.26907724, train_accuracy: 0.9609, test_Accuracy: 0.9378
Epoch: [ 0] [  229/  468] time: 60.9045, train_loss: 0.43192634, train_accuracy: 0.8984, test_Accuracy: 0.9373
Epoch: [ 0] [  230/  468] time: 61.1840, train_loss: 0.48757118, train_accuracy: 0.9219, test_Accuracy: 0.9388
E

Epoch: [ 0] [  296/  468] time: 78.0327, train_loss: 0.33642566, train_accuracy: 0.9219, test_Accuracy: 0.9462
Epoch: [ 0] [  297/  468] time: 78.2733, train_loss: 0.22499615, train_accuracy: 0.9531, test_Accuracy: 0.9462
Epoch: [ 0] [  298/  468] time: 78.5182, train_loss: 0.25357145, train_accuracy: 0.9688, test_Accuracy: 0.9464
Epoch: [ 0] [  299/  468] time: 78.7680, train_loss: 0.32835943, train_accuracy: 0.9375, test_Accuracy: 0.9464
Epoch: [ 0] [  300/  468] time: 79.0083, train_loss: 0.27371460, train_accuracy: 0.9766, test_Accuracy: 0.9469
Epoch: [ 0] [  301/  468] time: 79.2553, train_loss: 0.30216324, train_accuracy: 0.9219, test_Accuracy: 0.9462
Epoch: [ 0] [  302/  468] time: 79.5059, train_loss: 0.24569468, train_accuracy: 0.9453, test_Accuracy: 0.9464
Epoch: [ 0] [  303/  468] time: 79.7692, train_loss: 0.25431183, train_accuracy: 0.9531, test_Accuracy: 0.9465
Epoch: [ 0] [  304/  468] time: 80.0205, train_loss: 0.41460791, train_accuracy: 0.9531, test_Accuracy: 0.9456
E

Epoch: [ 0] [  370/  468] time: 99.2765, train_loss: 0.26966363, train_accuracy: 0.9531, test_Accuracy: 0.9532
Epoch: [ 0] [  371/  468] time: 99.5576, train_loss: 0.17956598, train_accuracy: 0.9609, test_Accuracy: 0.9532
Epoch: [ 0] [  372/  468] time: 99.8105, train_loss: 0.31011772, train_accuracy: 0.9453, test_Accuracy: 0.9533
Epoch: [ 0] [  373/  468] time: 100.0982, train_loss: 0.39855647, train_accuracy: 0.9453, test_Accuracy: 0.9538
Epoch: [ 0] [  374/  468] time: 100.3741, train_loss: 0.20565529, train_accuracy: 0.9688, test_Accuracy: 0.9539
Epoch: [ 0] [  375/  468] time: 100.7075, train_loss: 0.40041992, train_accuracy: 0.9219, test_Accuracy: 0.9547
Epoch: [ 0] [  376/  468] time: 100.9997, train_loss: 0.25827509, train_accuracy: 0.9297, test_Accuracy: 0.9551
Epoch: [ 0] [  377/  468] time: 101.4063, train_loss: 0.32506886, train_accuracy: 0.9766, test_Accuracy: 0.9552
Epoch: [ 0] [  378/  468] time: 101.6850, train_loss: 0.35738301, train_accuracy: 0.9375, test_Accuracy: 0.

Epoch: [ 0] [  444/  468] time: 119.9979, train_loss: 0.21151850, train_accuracy: 0.9766, test_Accuracy: 0.9565
Epoch: [ 0] [  445/  468] time: 120.2564, train_loss: 0.35397613, train_accuracy: 0.9297, test_Accuracy: 0.9544
Epoch: [ 0] [  446/  468] time: 120.5442, train_loss: 0.44879848, train_accuracy: 0.9297, test_Accuracy: 0.9528
Epoch: [ 0] [  447/  468] time: 120.8025, train_loss: 0.27594784, train_accuracy: 0.9453, test_Accuracy: 0.9510
Epoch: [ 0] [  448/  468] time: 121.0566, train_loss: 0.36467877, train_accuracy: 0.9375, test_Accuracy: 0.9513
Epoch: [ 0] [  449/  468] time: 121.2935, train_loss: 0.22918865, train_accuracy: 0.9766, test_Accuracy: 0.9528
Epoch: [ 0] [  450/  468] time: 121.5691, train_loss: 0.41348827, train_accuracy: 0.9297, test_Accuracy: 0.9547
Epoch: [ 0] [  451/  468] time: 121.8527, train_loss: 0.14517848, train_accuracy: 0.9531, test_Accuracy: 0.9561
Epoch: [ 0] [  452/  468] time: 122.1058, train_loss: 0.25073293, train_accuracy: 0.9609, test_Accuracy:

## Test Accuracy : 0.96589